### Building NGC graphs

In [6]:
import tensorflow as tf
from ngclearn.engine.nodes.enode import ENode
from ngclearn.engine.ngc_graph import NGCGraph
from construction_utils import SNodeBuilder, CableConnector
bld = SNodeBuilder()
cc = CableConnector()

Define nodes

In [7]:
a = bld.O1_set_numerals(beta=0.05, leak=0.001).O2_set_cats().O0_build("a", dim=20)
b = bld.O1_set_numerals(beta=0.05, leak=0.002).O2_set_cats().O0_build("b", dim=10)
e = ENode("e", dim=10)

Connect nodes

In [ ]:
a_e = cc.O1_dense().Op1_with_update_rule().Op2_with_constraints().O0_connect(a, e, to_comp=cc.EComps.PMU)
b_e = cc.O1_simple(1.0).O0_connect(b, e, from_comp=cc.SComps.Z, to_comp=cc.EComps.PTARG)
cc.O1_mirror(a_e).O0_connect(e, a, to_comp=cc.SComps.BU)
cc.O1_simple(-1.0).O0_connect(e, b)

Construct Graph


In [9]:
model = NGCGraph(K=5)
model.set_cycle([a,b])
model.set_cycle([e])
model.apply_constraints()
info = model.compile(batch_size=1)

Test the model on some ones


In [35]:
opt = tf.optimizers.SGD()
x = tf.ones([1, 10])

readouts = model.settle(
                clamped_vars=[("b","z",x)],
                readout_vars=[("a","phi(z)"),("e","phi(z)")]
)

print("The value of {} w/in Node {} is {}".format(readouts[0][0][1], readouts[0][0][0], readouts[0][0][2].numpy()))

delta = model.calc_updates()
opt.apply_gradients(zip(delta, model.theta)) # apply a TF optimizer here
model.apply_constraints()
model.clear()

readouts = model.settle(
                clamped_vars=[("b","z",x)],
                readout_vars=[("a","phi(z)"),("e","phi(z)")]
)

print("The value of {} w/in Node {} is {}".format(readouts[0][0][1], readouts[0][0][0], readouts[0][0][2].numpy()))


The value of phi(z) w/in Node a is [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
The value of phi(z) w/in Node a is [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
